# Bajando datos de diferentes fuentes. Ejemplos

## Bajar datos de Bicimad

In [1]:
import requests
import zipfile
import pandas as pd 

url = 'https://opendata.emtmadrid.es/getattachment/7517a650-ccdf-4ab1-b1b0-a1d13694472e/201906_Usage_Bicimad.aspx'

filename = 'bicis.zip'

r = requests.get(url, allow_redirects = True)

In [2]:
with open(filename, 'wb') as f: #wb=='write binary', abro el fichero y escribo en él
    # con with los recursos se cierran bien. para que no se quede abierto y limite para seguir a trabajando por no haber cerrado. Así es automático.
    f.write(r.content)

with zipfile.ZipFile(filename, 'r') as zip_ref: #r == 'read'
    zip_ref.extractall(".") #Se extraen todo aquí, en '.' == directorio actual

In [3]:
datos_bicis = pd.read_json('201906_Usage_Bicimad.json', lines=True, encoding= 'latin-1')

In [4]:
datos_bicis.head(3)

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,track
0,{'$oid': '5cf83b752f3843a016be4e2f'},e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e...,21,1,8,219,90,0,66,{'$date': '2019-06-01T00:00:00.000+0200'},,NaN
1,{'$oid': '5cf83b762f3843a016be4e48'},8a0c4123e924a50a958f51985eb71aea750fb072438035...,19,1,19,359,71,4,136,{'$date': '2019-06-01T00:00:00.000+0200'},28039,NaN
2,{'$oid': '5cf83b762f3843a016be4e4f'},a6a9c1f74a68496000542210abc4fc2eba79e2756ad535...,17,1,7,375,39,4,38,{'$date': '2019-06-01T00:00:00.000+0200'},28013,NaN


In [5]:
datos_bicis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450811 entries, 0 to 450810
Data columns (total 12 columns):
_id                 450811 non-null object
user_day_code       450811 non-null object
idplug_base         450811 non-null int64
user_type           450811 non-null int64
idunplug_base       450811 non-null int64
travel_time         450811 non-null int64
idunplug_station    450811 non-null int64
ageRange            450811 non-null int64
idplug_station      450811 non-null int64
unplug_hourTime     450811 non-null object
zip_code            450811 non-null object
track               115594 non-null object
dtypes: int64(7), object(5)
memory usage: 41.3+ MB


## Importando datos desde la API de INE

In [6]:
url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'

# codigo de la serie de datos a consultar y numero de datos
codigo ='IPC118150' #Datos de variación del IPC en andalucia
30013
num_datos = 12 #últimos doce datos

url = url_plantilla.format(codigo=codigo, num_datos=num_datos)
# realizar la descarga de los datos usando la libreria request, y leyendo el formato json
respuesta = requests.get(url)
datos = respuesta.json()
print(datos)

# obtenemos el nombre de la serie para nombrar la columna en el data frame
nombre = datos['Nombre']

# Creamos una serie con las fechas, y las convertimos a zona horaria española y formateamos
fecha_serie_utc = pd.to_datetime([x['Fecha'] for x in datos['Data']], unit='ms', utc= True)

# en x la columna Fecha sale del valor en datos['Data'] pero formateando a fecha
fecha_serie_madrid = fecha_serie_utc.tz_convert('Europe/Madrid')

fecha_serie = [x.tz_localize(None).date() for x in fecha_serie_madrid]


# creamos una lista con los valores de la serie que vienen en el tag 'Valor'

ocupados_serie = [x['Valor'] for x in datos['Data']]

print(type(ocupados_serie))

{'COD': 'IPC118150', 'Nombre': 'Andalucía, Total, Base 2011, General, Variación mensual', 'FK_Unidad': 135, 'FK_Escala': 1, 'Data': [{'Fecha': 1451602800000, 'FK_TipoDato': 1, 'FK_Periodo': 1, 'Anyo': 2016, 'Valor': -2.0, 'Secreto': False}, {'Fecha': 1454281200000, 'FK_TipoDato': 1, 'FK_Periodo': 2, 'Anyo': 2016, 'Valor': -0.4, 'Secreto': False}, {'Fecha': 1456786800000, 'FK_TipoDato': 1, 'FK_Periodo': 3, 'Anyo': 2016, 'Valor': 0.8, 'Secreto': False}, {'Fecha': 1459461600000, 'FK_TipoDato': 1, 'FK_Periodo': 4, 'Anyo': 2016, 'Valor': 0.8, 'Secreto': False}, {'Fecha': 1462053600000, 'FK_TipoDato': 1, 'FK_Periodo': 5, 'Anyo': 2016, 'Valor': 0.6, 'Secreto': False}, {'Fecha': 1464732000000, 'FK_TipoDato': 1, 'FK_Periodo': 6, 'Anyo': 2016, 'Valor': 0.4, 'Secreto': False}, {'Fecha': 1467324000000, 'FK_TipoDato': 1, 'FK_Periodo': 7, 'Anyo': 2016, 'Valor': -0.9, 'Secreto': False}, {'Fecha': 1470002400000, 'FK_TipoDato': 1, 'FK_Periodo': 8, 'Anyo': 2016, 'Valor': 0.1, 'Secreto': False}, {'Fecha'

In [7]:
# contruimos un dataframe de pandas con los valores

tabla = pd.DataFrame(ocupados_serie, index=fecha_serie, columns=[nombre])
tabla

,"Andalucía, Total, Base 2011, General, Variación mensual"
2016-01-01,-2.0
2016-02-01,-0.4
2016-03-01,0.8
2016-04-01,0.8
2016-05-01,0.6
2016-06-01,0.4
2016-07-01,-0.9
2016-08-01,0.1
2016-09-01,0.2
2016-10-01,1.2


## Web scrapping de IMDB

WEB_SCRAPPIN, es un proceso automático(tipo robot-araña recorre la web) con python, através de sus librerías, para detectar elementos ... 'rascando' en internet.

In [8]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 

# Download IMDB's Top 250 data

url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

imdb = []

# Store each item into dictionary (data), then put those into a list (imdb)

for index in range(0, len(movies)):
    # Seperate movie into: 'place', 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
            "year": year,
            "place": place,
            "star_cast": crew[index],
            "rating": ratings[index],
            "vote": votes[index],
            "link": links[index]}
    imdb.append(data)
df = pd.DataFrame(imdb)
df


#for item in imdb:
#    print(item['place'], '-', item['movie_title'], '('+item['year']+') -', 'Starring:', item['star_cast'])

,movie_title,year,place,star_cast,rating,vote,link
0,Cadena perpetua,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.222777188736787,None,/title/tt0111161/
1,El padrino,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.148708938669726,None,/title/tt0068646/
2,El padrino: Parte II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980959295423688,None,/title/tt0071562/
3,El caballero oscuro,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.973507640978838,None,/title/tt0468569/
4,12 hombres sin piedad,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.93076327560344,None,/title/tt0050083/
...,...,...,...,...,...,...,...
245,Terminator,1984,246,"James Cameron (dir.), Arnold Schwarzenegger, L...",8.009028394816163,None,/title/tt0088247/
246,Aladdín,1992,247,"Ron Clements (dir.), Scott Weinger, Robin Will...",8.007549926555294,None,/title/tt0103639/
247,Mandarinas,2013,248,"Zaza Urushadze (dir.), Lembit Ulfsak, Elmo Nüg...",8.007547658603647,None,/title/tt2991224/
248,A Silent Voice,2016,249,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.007273336080628,None,/title/tt5323662/


## Accediendo a datos de twitter

In [9]:
#!pip install tweepy 

import tweepy
import time
import csv

# estos token y keys hay que generarlos desde Twitter: https://apps.twitter.com
# https://developer.twitter.com/
# hay que registrarse una cuenta de desarrollador de aplicaciones que usan Twitter
# esos token y keys caducan pasado un tiempo (nos avisa al crearlos)

access_token = "341758650-EHFbLBVbH4cgecuSU5gzRY19VPo9W4VZPWSDdY3a"  
access_token_secret = "nX7FA4I0GfDxKwdQfhOxY8ylMIC4mzDGfApCIEJuMVWLM"  
consumer_key = "dyTOIkKEeJDXuIlI8fNhT8e91" 
consumer_secret = "rMPGd7X4gFzjxS8lYJ8GagwHAGG4bw3fqJZb9MVVhb4HN2ICnI"
 

In [10]:
''' https://realpython.com/twitter-bot-python-tweepy/'''

' https://realpython.com/twitter-bot-python-tweepy/'

In [11]:
#Authenticate to twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [12]:
csvFile = open('cop25.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search, q='#cop25', count=5000, lang= 'en', since='2019-1-12').items():
    print(tweet.created_at, tweet)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

ground_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1266664463542947840/1JHE0GBT_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1266664463542947840/1JHE0GBT_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/326665662/1356119193', 'profile_link_color': '0084B4', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'ret

In [19]:
csvFile = open('enlightED.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search, q='#enlightED', count=5000, lang= 'es', since='2019-1-12').items():
    print(tweet.created_at, tweet)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

_images/911103959296704515/a9sYPu0t_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/911103959296704515/a9sYPu0t_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/216065172/1591501185', 'profile_link_color': '1B95E0', 'profile_sidebar_border_color': 'FAF8F7', 'profile_sidebar_fill_color': 'FAF7F5', 'profile_text_color': '593A1A', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 6, 'favorite_count': 7, 'favorited': False, 'retweeted': False, 'possibly_sensitive': False, 'lang': 'es'}, created_at=datetime.datetime(2020, 10, 14, 17, 42, 14), id=1316434151726297088, id_str='1316434151726297088', text='¿QUIERES FORMAR PARTE DE LAS REFLEXIONES QUE PLAN

In [7]:
#Use csv Writer


## Bajando datos de la calidad del aire

También existe un dataset en Kaggle, pero vamos a ver un ejemplo

In [14]:
import pandas as pd 
import requests
import io 
import csv

In [15]:
url2 = "http://www.mambiente.munimadrid.es/opendata/horario.txt"

datos= pd.read_csv(url2, header=None)


In [16]:
datos.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,28,79,4,1,38,2,2020,10,22,2.0,...,0,N,0,N,0,N,0,N,0,N
1,28,79,4,6,48,2,2020,10,22,0.2,...,0,N,0,N,0,N,0,N,0,N
2,28,79,4,7,8,2,2020,10,22,2.0,...,0,N,0,N,0,N,0,N,0,N
3,28,79,4,8,8,2,2020,10,22,14.0,...,0,N,0,N,0,N,0,N,0,N
4,28,79,4,12,8,2,2020,10,22,16.0,...,0,N,0,N,0,N,0,N,0,N
